In [1]:
-

In [2]:
call_record=pd.read_csv('chapter1data/net_rel.csv',index_col=0)
call_record=call_record[call_record.custorm_id!=call_record.cust_b]
g=getGraph(call_record.values)

/opt/anaconda2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
MaxScc=g.decompose(maxcompno=True)[0]   ##最大联通分量
names=MaxScc.vs['name']
## 可以调节度的大小进行缩放网络
min_degree=30
v3=MaxScc.vs.select(_degree_gt=min_degree)   ## 选择度大于1的节点并组成子图 ；尽量去除那些叶子节点
rel_net=MaxScc.subgraph(v3)               ## 并构成子图求出子图中的最大联通分量
rel_net=rel_net.decompose(maxcompno=True)[0]
print '节点个数：',rel_net.vcount()
print '边的个数:',rel_net.ecount()
print '图密度:',rel_net.density()
# print '直径：',rel_net.diameter()
# print '聚集系数：',rel_net.transitivity_undirected()

节点个数： 67140
边的个数: 291961
图密度: 0.000129538431118


In [10]:
%time relW_comm=rel_net.community_label_propagation(weights='weight')

CPU times: user 1.4 s, sys: 0 ns, total: 1.4 s
Wall time: 1.4 s


In [20]:
relW_comm.compare_to()  ##比较两种社区划分的效果,0表示相似

0.0

In [32]:
max_comm=relW_comm.giant()

In [35]:
max_comm_clu=max_comm.community_label_propagation()

In [38]:
max_comm_clu.summary()

'Clustering with 2584 elements and 7 clusters'

In [51]:
comm_cover=max_comm_clu.as_cover()

In [132]:
df=pd.DataFrame()
for i in range(7):
    c=comm_cover.subgraph(i).vs['name']
    df=pd.concat([df,pd.DataFrame(np.array([c,[i]*len(c)])).T],axis=0)
## 写入顶点
f=open('dd.net','w')
f.write('*Vertices '+str(df.shape[0])+'\n')
for line in df.values:
    f.write(str(line[0])+' "'+str(line[1])+'"\n')

f.write('*Edges'+'\n')
edges=pd.DataFrame([list(line.tuple)  for line in  list(max_s.es)])
edges['weight']=max_s.es['weight']
for line in edges.values:
    f.write(str(int(line[0]))+' '+str(int(line[1]))+' '+str(line[2])+'\n')
f.close()


In [146]:
import igraph  as ig

In [148]:
ig.compare_communities()   ##比较社区结构

In [154]:
ig.Graph.union




In [244]:
subg1=max_comm_clu.subgraph(1)
subg1.vs['clu']=['1']*85

In [261]:
subg1.vcount()

85

In [245]:
for i in range(2,6):
    subg2=max_comm_clu.subgraph(i)
    subg2.es['clu']=[str(i)]*subg2.vcount()
    x=subg1.union(subg2)
    
# subg2.vs['clu']=['1']*85

In [260]:
subg2.vcount()

8

In [204]:
x=subg2.union(max_comm_clu.subgraph(3))

In [222]:
x=[IGraph.union(line,subg)  for line in max_comm_clu.subgraphs()[1:]]

In [225]:
subg2.vs['name']

[1224125, 43261, 4217713, 1310798, 208002, 1748808, 211406, 1010473]

In [247]:
x=subg1.union(subg2)

In [259]:
pose(subg2)
x.vcount()

85

In [346]:
subg.community_infomap().summary()

'Clustering with 85 elements and 22 clusters'

In [188]:
# subg.write_svg('sub.svg')
subg.write_graphml('subg.graphml')

In [107]:
edges.to_excel('dd.xls')